In [13]:
# Imports

import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, GRU
from tensorflow.keras.callbacks import EarlyStopping


In [14]:
# select the stock from yfinance

ticker = 'AAPL'
data = yf.download(ticker, start='2015-01-01', end='2024-01-01')
close_prices = data[['Close']]

[*********************100%***********************]  1 of 1 completed


In [15]:
# data splits and preprocessing

scaler = MinMaxScaler(feature_range=(0, 1))
prices_scaled = scaler.fit_transform(close_prices.values)

def create_dataset(dataset, time_step=60):
    X, y = [], []
    for i in range(len(dataset) - time_step):
        X.append(dataset[i:i + time_step, 0])
        y.append(dataset[i + time_step, 0])
    return np.array(X), np.array(y)

time_step = 60
X, y = create_dataset(prices_scaled, time_step)
X = X.reshape(X.shape[0], X.shape[1], 1)

train_size = int(len(X) * 0.8)
x_train, x_val = X[:train_size], X[train_size:]
y_train, y_val = y[:train_size], y[train_size:]

In [ ]:
# Show the raw stock data graph

plt.figure(figsize=(12, 5))
plt.plot(close_prices, label=f'{ticker} Closing Price')
plt.title(f'{ticker} Closing Price History')
plt.xlabel('Date')
plt.ylabel('Price (USD)')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# GRU model for the price prediction

gru_model = Sequential([
    GRU(50, return_sequences=True, input_shape=(time_step, 1)),
    GRU(50),
    Dense(25),
    Dense(1)
])
gru_model.compile(optimizer='adam', loss='mean_squared_error')
gru_model.summary()

early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
history = gru_model.fit(
    x_train, y_train,
    validation_data=(x_val, y_val),
    epochs=50,
    batch_size=32,
    callbacks=[early_stop],
    verbose=1
)



/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ gru_4 (GRU)                          │ (None, 60, 50)              │           7,950 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_5 (GRU)                          │ (None, 50)                  │          15,300 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (None, 25)                  │           1,275 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ (None, 1)                   │              26 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 24,551 (95.90 KB)

 Trainable params: 24,551 (95.90 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 11s 108ms/step - loss: 0.0288 - val_loss: 5.9594e-04
Epoch 2/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 9s 81ms/step - loss: 1.9239e-04 - val_loss: 4.8316e-04
Epoch 3/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 5s 90ms/step - loss: 1.4052e-04 - val_loss: 4.9640e-04
Epoch 4/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 4s 76ms/step - loss: 1.6818e-04 - val_loss: 4.3134e-04
Epoch 5/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 7s 101ms/step - loss: 1.2844e-04 - val_loss: 4.1193e-04
Epoch 6/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 4s 76ms/step - loss: 1.1814e-04 - val_loss: 3.8742e-04
Epoch 7/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 5s 75ms/step - loss: 1.1451e-04 - val_loss: 3.6266e-04
Epoch 8/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 6s 90ms/step - loss: 1.1385e-04 - val_loss: 4.4747e-04
Epoch 9/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 10s 95ms/step - loss: 1.1376e-04 - val_loss: 3.3692e-04
Epoch 10/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 9s 74ms/step - loss: 9.8572e-05 - val_loss: 4.3630e-04
Epoch 11/50
19/56 ━━━━━━━━━━━━━━━━━━━━ 2s 75ms/step - loss: 1.1410e-0

In [ ]:
# Show the GRU graph

gru_x_val=x_val
gru_y_val=y_val
gru_y_pred_scaled = gru_model.predict(gru_x_val)
gru_y_pred = scaler.inverse_transform(gru_y_pred_scaled)
y_actual = scaler.inverse_transform(y_val.reshape(-1, 1))

plt.figure(figsize=(12, 6))
plt.plot(y_actual, label='Actual Price')
plt.plot(gru_y_pred, label='Predicted Price')
plt.title(f'{ticker} Price Prediction vs Actual (Validation Set) - GRU')
plt.xlabel('Time')
plt.ylabel('Price (USD)')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Error metrics for GRU

gru_rmse = np.sqrt(mean_squared_error(y_actual, gru_y_pred))
gru_mae = mean_absolute_error(y_actual, gru_y_pred)
gru_r2 = r2_score(y_actual, gru_y_pred)
print(f'Validation RMSE: {gru_rmse:.4f} USD')
print(f'Validation MAE: {gru_mae:.4f} USD')
print(f'Validation R²: {gru_r2:.4f}')

In [ ]:
# LSTM for the same price prediction thiing

lstm_model = Sequential([
    LSTM(50, return_sequences=True, input_shape=(time_step, 1)),
    LSTM(50),
    Dense(25),
    Dense(1)
])
lstm_model.compile(optimizer='adam', loss='mean_squared_error')
lstm_model.summary()

early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
history = lstm_model.fit(
    x_train, y_train,
    validation_data=(x_val, y_val),
    epochs=50,
    batch_size=32,
    callbacks=[early_stop],
    verbose=1
)

In [ ]:
# Show the LSTM graph

lstm_x_val=x_val
lstm_y_val=y_val
lstm_y_pred_scaled = lstm_model.predict(lstm_x_val)
lstm_y_pred = scaler.inverse_transform(lstm_y_pred_scaled)
y_actual = scaler.inverse_transform(y_val.reshape(-1, 1))


plt.figure(figsize=(12, 6))
plt.plot(y_actual, label='Actual Price')
plt.plot(lstm_y_pred, label='Predicted Price')
plt.title(f'{ticker} Price Prediction vs Actual (Validation Set)')
plt.xlabel('Time')
plt.ylabel('Price (USD)')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
# Error metrics for LSTM

lstm_rmse = np.sqrt(mean_squared_error(y_actual, lstm_y_pred))
lstm_mae = mean_absolute_error(y_actual, lstm_y_pred)
lstm_r2 = r2_score(y_actual, lstm_y_pred)
print(f'Validation RMSE: {lstm_rmse:.4f} USD')
print(f'Validation MAE: {lstm_mae:.4f} USD')
print(f'Validation R²: {lstm_r2:.4f}')

In [ ]:
# Comparision plot between gru and lstm

y_lstm_pred = scaler.inverse_transform(lstm_model.predict(x_val))
y_gru_pred = scaler.inverse_transform(gru_model.predict(x_val))
y_actual = scaler.inverse_transform(y_val.reshape(-1, 1))

plt.figure(figsize=(14, 6))
plt.plot(y_actual, label='Actual Price', linewidth=2)
plt.plot(y_lstm_pred, label='LSTM Prediction')
plt.plot(y_gru_pred, label='GRU Prediction')
plt.title('Stock Price Prediction: LSTM vs GRU')
plt.xlabel('Time')
plt.ylabel('Price (USD)')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# comparision of metrics
lstm_rmse = np.sqrt(mean_squared_error(y_actual, lstm_y_pred))
gru_rmse = np.sqrt(mean_squared_error(y_actual, lstm_y_pred))

lstm_mae = mean_absolute_error(y_actual, lstm_y_pred)
gru_mae = mean_absolute_error(y_actual, gru_y_pred)

lstm_r2 = r2_score(y_actual, lstm_y_pred)
gru_r2 = r2_score(y_actual, gru_y_pred)

# Display results
print(f"{'Model':<10} {'RMSE':>10} {'MAE':>10} {'R² Score':>12}")
print(f"{'-'*40}")
print(f"{'LSTM':<10} {lstm_rmse:10.4f} {lstm_mae:10.4f} {lstm_r2:12.4f}")
print(f"{'GRU':<10} {gru_rmse:10.4f} {gru_mae:10.4f} {gru_r2:12.4f}")

# Compare
print("\n🔍 Performance Comparison:")
if lstm_rmse < gru_rmse:
    print("✅ LSTM has lower RMSE.")
else:
    print("✅ GRU has lower RMSE.")

if lstm_mae < gru_mae:
    print("✅ LSTM has lower MAE.")
else:
    print("✅ GRU has lower MAE.")

if lstm_r2 > gru_r2:
    print("✅ LSTM has higher R² (better explained variance).")
else:
    print("✅ GRU has higher R² (better explained variance).")



In [ ]:
# for the csv
results_df = pd.DataFrame({
    'Date': close_prices.index[-len(y_actual):],  # align with validation window
    'Actual_Price': y_actual.flatten(),
    'LSTM_Predicted': lstm_y_pred.flatten(),
    'GRU_Predicted': gru_y_pred.flatten()
})

# Ensure 'Date' is the first column
results_df.set_index('Date', inplace=True)

# Display first few rows
print(results_df.head())

# Save to CSV
results_df.to_csv(f'{ticker}_lstm_gru_predictions.csv')
print(f"\n✅ Saved predictions to '{ticker}_lstm_gru_predictions.csv'")
